In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain
!pip install -qU langchain-huggingface
!pip install --upgrade --quiet  langchain sentence_transformers
!pip install tf-keras
!pip install langchain-ollama
!pip install langchain-huggingface

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [ ]:
# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

print(type(docs))
print(len(docs))
print(docs[0].metadata)
docs[0].page_content[:200]

<class 'list'>
1
{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}


'\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a '

In [ ]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

print(type(splits), len(splits))
print(len(splits[0].page_content), '\n', splits[0].page_content[:200])

<class 'list'> 66
969 
 LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool con


In [ ]:
## Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embeddings = OllamaEmbeddings(model="llama3")
# embeddings = OpenAIEmbeddings()

# Test embeddings
text = 'I am robot'
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])

print(type(query_result), type(doc_result))
print(len(query_result), query_result[:5])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<class 'list'> <class 'list'>
768 [0.03698704391717911, -0.0025559028144925833, -0.018150417134165764, -0.024968178942799568, 0.037337612360715866]


In [ ]:
#### INDEXING
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

print(type(vectorstore), type(retriever))
display(dict(retriever))

<class 'langchain_community.vectorstores.chroma.Chroma'> <class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [ ]:
retriever.get_relevant_documents('vector data store')

<ipython-input-19-06736a7135c7>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever2.get_relevant_documents('vector data store')


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Maximum Inner Product Search (MIPS)#\nThe external memory can alleviate the restriction of finite attention span.  A standard practice is to save the embedding representation of information into a vector store database that can support fast maximum inner-product search (MIPS). To optimize the retrieval speed, the common choice is the approximate nearest neighbors (ANN)\u200b algorithm to return approximately top k nearest neighbors to trade off a little accuracy lost for a huge speedup.\nA couple common choices of ANN algorithms for fast MIPS:'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Maximum Inner Product Search (MIPS)#\nThe external memory can alleviate the restriction of finite attention span.  A standard practice is to save the embedding representation of information into a vector store database that can support fast maxi

In [ ]:
retriever.invoke('super man')

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Maximum Inner Product Search (MIPS)#\nThe external memory can alleviate the restriction of finite attention span.  A standard practice is to save the embedding representation of information into a vector store database that can support fast maximum inner-product search (MIPS). To optimize the retrieval speed, the common choice is the approximate nearest neighbors (ANN)\u200b algorithm to return approximately top k nearest neighbors to trade off a little accuracy lost for a huge speedup.\nA couple common choices of ANN algorithms for fast MIPS:'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Maximum Inner Product Search (MIPS)#\nThe external memory can alleviate the restriction of finite attention span.  A standard practice is to save the embedding representation of information into a vector store database that can support fast maxi

In [ ]:
## Prompt template defined
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

In [ ]:
## Prompt template from langchain hub
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
## LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

llm.invoke('Where is Paris?')

AIMessage(content='Paris is the capital city of France and is located in the northern-central part of the country.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 11, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ea84ee53-ece6-46df-97f1-10a659c2e6e7-0', usage_metadata={'input_tokens': 11, 'output_tokens': 19, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
## Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Question
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a problem into multiple thought steps and generating multiple thoughts per step, creating a tree structure. It can be achieved through methods like using LLM with simple prompting, task-specific instructions, or human inputs. Tree of Thoughts extends this concept by exploring multiple reasoning possibilities at each step.'

In [ ]:
## Other way of creating model pipeline
from langchain.llms import HuggingFaceHub
from huggingface_hub import list_models
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
)
hf = HuggingFacePipeline(pipeline=pipe)


# Initialize the model
# gpt2_model = HuggingFaceHub(
#     hub_id="gpt2",
#     model_kwargs={"temperature": 0.7}
# )


models = list_models()
display(next(models))

hf.invoke('What is capital of india')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'What is capital of india?\u202c\n\n\n\n\n\n\n'

In [ ]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat. And I hate Dogs as well as Parrots."

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embd = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)

print(len(query_result))
print(query_result[:5])

768
[0.01589711382985115, 0.062301717698574066, -0.037331655621528625, -0.03464021161198616, 0.032484021037817]


In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity using numpy:", similarity)

Cosine Similarity using numpy: 0.5979315079774208


In [ ]:
template1 = """You are an AI language model assistant.
Ask below question in three deifferent ways. Keep questions short.

Original question: {question}"""
prompt1 = ChatPromptTemplate.from_template(template1)

x1 = (
    prompt1
    | ChatOpenAI(temperature=0)
)

x1.invoke('Where is Paris in World??')

AIMessage(content='1. What is the location of Paris in the world?\n2. In which part of the world is Paris located?\n3. Where can Paris be found in the world?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 38, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e58b940b-fad9-4231-bbb4-92fa07a719ff-0', usage_metadata={'input_tokens': 38, 'output_tokens': 35, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
template2 = """You are an AI language model assistant.
Ask below question in three deifferent ways. Keep questions short.

Original question: {question}"""
prompt2 = ChatPromptTemplate.from_template(template2)

x2 = (
    prompt2
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)
x2.invoke('Where is Paris in World??')

'1. What is the location of Paris in the world?\n2. In which part of the world is Paris located?\n3. Where can Paris be found in the world?'

In [ ]:
template3 = """You are an AI language model assistant.
Ask below question in three deifferent ways. Keep questions short.

Original question: {question}"""
prompt3 = ChatPromptTemplate.from_template(template3)

x3 = (
    prompt3
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)
x3.invoke('Where is Paris in World??')

['1. What is the location of Paris in the world?',
 '2. In which part of the world is Paris located?',
 '3. Where can Paris be found in the world?']

# <mark> PART 5 - 9 where Advanced Query transformation techniques are employed

#### <mark> 5. Multi Query: Different Perspectives

In [ ]:
## Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries_chain = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

generate_queries_chain.invoke('What is task decomposition for LLM agents?')

['1. How do LLM agents utilize task decomposition?',
 '2. Can you explain the concept of task decomposition in relation to LLM agents?',
 '3. What role does task decomposition play in the functioning of LLM agents?',
 '4. How is task decomposition implemented by LLM agents?',
 '5. In what way does task decomposition benefit LLM agents?']

In [ ]:
retriever.map()

RunnableEach(bound=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f4a871b56c0>, search_kwargs={}))

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve unique documents retrieved by multiple queries generated from originla question
question = "What is task decomposition for LLM agents?"
retrieval_chain = (
    generate_queries_chain
    | retriever.map()
    | get_unique_union
)

##
docs = retrieval_chain.invoke({"question":question})
len(docs)

1

In [ ]:
from operator import itemgetter

final_rag_chain = (
    {"context": retrieval_chain, ##This chain return unique document retrived from vectorstore
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents involves breaking down complex tasks into smaller and simpler steps using techniques like Chain of thought (CoT), which prompts the model to "think step by step" in order to utilize more test-time computation and make the tasks more manageable.'

#### <mark> 6. RAG fusion

In [ ]:
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n

Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [ ]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = (
    generate_queries
    | retriever.map()
    | reciprocal_rank_fusion
    )
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

1

In [ ]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents involves breaking down complex tasks into smaller and simpler steps using the Chain of thought (CoT) prompting technique. This allows the agent to utilize more test-time computation and enhance its performance on complex tasks by transforming big tasks into multiple manageable tasks.'

### <mark> 7. SubQuestion (Least-to-Most) Decomposition

In [ ]:
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [ ]:
# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})
questions

['1. What is LLM technology and how does it work in autonomous agent systems?',
 '2. What are the specific components that make up an LLM-powered autonomous agent system?',
 '3. How do the main components of an LLM-powered autonomous agent system interact with each other to enable autonomous behavior?']

Answer recursively

In [ ]:
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [ ]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()


q_a_pairs = ""
for q in questions:

    rag_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")}
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [ ]:
answer

'The main components of an LLM-powered autonomous agent system, namely Planning and Task Decomposition, interact with each other to enable autonomous behavior. \n\n1. Planning: This component involves understanding the steps required in a complex task and planning ahead to execute them effectively. By utilizing techniques like Chain of Thought (CoT), the agent can break down the task into smaller, more manageable steps. Planning helps the agent to have a clear roadmap of what needs to be done and in what sequence.\n\n2. Task Decomposition: Task Decomposition, facilitated by techniques like CoT, prompts the model to "think step by step" and decompose hard tasks into simpler and more manageable steps. This process transforms big tasks into multiple smaller tasks, making it easier for the agent to tackle complex tasks effectively. Task Decomposition complements Planning by breaking down the overall task into actionable steps that the agent can follow systematically.\n\nThe interaction bet

Answer individually


In [ ]:
## Instead of recursively passing pair of question-answer to next question in sequence,
# we can concatenate answers of all decomposed question and pass it along with context to get final answer.

# Prompt example
template_indivisual = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

<mark> 8 Step Back

In [ ]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]

## Now create necessary prompts
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
display(example_prompt, '\n')

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
display(few_shot_prompt, '\n')

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge.
            Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer.
            Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)
display(prompt, '\n')

ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})])

'\n'

FewShotChatMessagePromptTemplate(examples=[{'input': 'Could the members of The Police perform lawful arrests?', 'output': 'what can the members of The Police do?'}, {'input': 'Jan Sindel’s was born in what country?', 'output': 'what is Jan Sindel’s personal history?'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))

'\n'

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'Could the members of The Police perform lawful arrests?', 'output': 'what can the members of The Police do?'}, {'input': 'Jan Sindel’s was born in what country?', 'output': 'what is Jan Sindel’s personal history?'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}

'\n'

In [ ]:
generate_queries_step_back = (
    prompt
    | llm
    | StrOutputParser()
    )

sample_question = 'Which is popular breakfast snack in india?'
display(generate_queries_step_back.invoke({"question": sample_question}))

question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'What are common breakfast snacks in India?'

'What is the process of breaking down tasks for LLM agents?'

In [ ]:
# Response prompt
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)
response_prompt

ChatPromptTemplate(input_variables=['normal_context', 'question', 'step_back_context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['normal_context', 'question', 'step_back_context'], input_types={}, partial_variables={}, template='You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.\n\n# {normal_context}\n# {step_back_context}\n\n# Original Question: {question}\n# Answer:'), additional_kwargs={})])

In [ ]:
from langchain_core.runnables import RunnableLambda
chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

question1 = 'What is Chain of thoughts in generative ai application?'
chain.invoke({"question": question1})

'Chain of Thoughts (CoT) is a concept in generative AI applications that involves exploring multiple reasoning possibilities at each step. It decomposes a problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be conducted using either breadth-first search (BFS) or depth-first search (DFS), with each state evaluated by a classifier or through a majority vote.\n\nTask decomposition in CoT can be achieved in several ways. One method is through Large Language Models (LLM) with simple prompting, such as asking for the steps or subgoals for achieving a specific task. Another approach involves using task-specific instructions, like requesting a story outline for writing a novel. Additionally, human inputs can also be utilized for task decomposition in CoT.\n\nOverall, Chain of Thoughts in generative AI applications like CoT allows for the exploration of multiple reasoning paths and the generation of diverse thoughts to 

<mark> Part 9: HyDE (Hypothetical Documents Embeddings)

In [ ]:
# HyDE document genration
template = """Please write a scientific paper passage of maximum 50 words to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

question = 'Does generativeAI technique addresses issues in LLMs using advanced RAG?'

In [ ]:
generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser()
)

hypothetical_document = generate_docs_for_retrieval.invoke(question)
hypothetical_document

'Generative AI techniques, such as the advanced Retrieval-Augmented Generation (RAG) model, have shown promise in addressing issues in Large Language Models (LLMs). By incorporating a retrieval mechanism, RAG can improve the quality and relevance of generated text, making it a valuable tool for enhancing LLM performance.'

In [ ]:
# Retrieve documents closer to hypothetical document from vectorstore/embeddings
retrieval_chain = generate_docs_for_retrieval | retriever
retireved_docs = retrieval_chain.invoke({"question":question})
len(retireved_docs), retireved_docs[0]

(4,
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='[13] Schick et al. “Toolformer: Language Models Can Teach Themselves to Use Tools.” arXiv preprint arXiv:2302.04761 (2023).\n[14] Weaviate Blog. Why is Vector Search so fast? Sep 13, 2022.\n[15] Li et al. “API-Bank: A Benchmark for Tool-Augmented LLMs” arXiv preprint arXiv:2304.08244 (2023).\n[16] Shen et al. “HuggingGPT: Solving AI Tasks with ChatGPT and its Friends in HuggingFace” arXiv preprint arXiv:2303.17580 (2023).\n[17] Bran et al. “ChemCrow: Augmenting large-language models with chemistry tools.” arXiv preprint arXiv:2304.05376 (2023).\n[18] Boiko et al. “Emergent autonomous scientific research capabilities of large language models.” arXiv preprint arXiv:2304.05332 (2023).\n[19] Joon Sung Park, et al. “Generative Agents: Interactive Simulacra of Human Behavior.” arXiv preprint arXiv:2304.03442 (2023).\n[20] AutoGPT. https://github.com/Significant-Gravitas/Auto-GPT\n[21] GPT

In [ ]:
# RAG
template = """Answer the following question based on this context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'Yes, the generative AI technique addresses issues in Large Language Models (LLMs) using advanced RAG (Retrieval-Augmented Generation) techniques. This can be inferred from the references to "Generative Agents: Interactive Simulacra of Human Behavior" in the provided context.'